In [1]:
import mpc4quantum as m4q
from tests import *

import numpy as np
import qutip as qt
from scipy.linalg import block_diag
from unittest import TestCase
import qutip_qip
from qutip_qip import operations

# Diagnostic imports
import os
import matplotlib.pyplot as plt
import matplotlib as mpl
cmap = plt.get_cmap('tab10')

# Set root for saving plots
rootname = '2021_09_13_OSQP'

# Default args for plot_operator
imshow_args = {'norm': mpl.colors.SymLogNorm(vmin=-1, vmax=1, linthresh=1e-3), 'cmap': plt.get_cmap('RdBu_r')}

def plot_operator(A, dim_x, args=imshow_args):
    dim_l = A.shape[1]//A.shape[0]
    fig, axes = plt.subplots(2, dim_l)
    fig.subplots_adjust(hspace=0)
    for i in range(dim_l):
        Ai = A.reshape(dim_x, -1, dim_x)[:, i, :]
        ax = axes[0, i]
        _ = ax.imshow(Ai.real, **args)
        ax = axes[1, i]
        im = ax.imshow(Ai.imag, **args)
    for ax in axes.flatten():
        ax.set_xticks([])
        ax.set_yticks([])
    fig.subplots_adjust(right=0.8, hspace=0)
    cbar_ax = fig.add_axes([0.85, 0.15, 0.05, 0.7])
    fig.colorbar(im, cax=cbar_ax)
    return fig, axes

In [2]:
class TestGateSynth(TestCase):
    def test_NOT_gate(self):
        for order in range(1, 5):
            # Parameters
            sat = 1
            du = .25
            clock = m4q.StepClock(dt=0.05, horizon=15, n_steps=50)

            # Experiment
            freqs = {'wQ': np.pi, 'wD': np.pi, 'wR': np.pi}
            qubit = RWA_Qubit(**freqs)
            gate_synth = m4q.QSynthesis(qubit.H_list[0], [qubit.H_list[1]])
            # Add this line to increase the solver's internal steps
            gate_synth.set('options', qt.Options(nsteps=5000))

            # Model
            
            As_cts_list = [-1j * np.kron(np.kron(h.full(), np.identity(2)) - np.kron(np.identity(2), h.full().conj()), np.identity(4)) for h in qubit.H_list] 
            A_init = m4q.discretize_homogeneous(As_cts_list, clock.dt, order)

            # DMDc object
            dim_lift = len(m4q.create_library(order, qubit.dim_u)[1:])
            dim_process = qubit.dim_x ** 2
            model1 = m4q.DMDc(dim_process, dim_process, dim_process * dim_lift, A_init)

            # Operations
            Rx = qutip_qip.operations.rx(1e-3)
            U0 = Rx * qt.identity(2)
            Uf = qt.sigmax()
            p0 = qt.tensor(U0, U0.conj()).full().flatten()
            pf = qt.tensor(Uf, Uf.conj()).full().flatten()

            # Benchmarks
            P_bm = np.hstack([pf.reshape(-1, 1)] * (clock.horizon + 1))
            U_bm = np.hstack([np.ones([qubit.dim_u, 1]) * .5] * clock.horizon)

            # Cost
            Q = np.identity(len(p0))
            Qf = Q * 1e1
            R = np.identity(qubit.dim_u) * 1e-2

            # MPC
            def exit_condition(p2, p1, u1):
                return ((p1 - pf).conj().T @ Q @ (p1 - pf)).real < 1e-2

            data, model2, exit_code = m4q.mpc(p0, qubit.dim_u, order, P_bm, U_bm, clock, gate_synth, model1, Q, R, Qf,
                                              sat=sat, du=du, exit_condition=exit_condition)
            ps, us = data

            # Save diagnostics
            path = './{}_NOT_gate/sat_{}_du_{}_{}/'.format(rootname, sat, du, clock.to_string())
            if not os.path.exists(path):
                os.makedirs(path)
            transparent = False
            if np.any(us):
                tsp1 = np.hstack([clock.ts_sim, clock.ts_sim[-1] + clock.dt])

                fig, ax = plt.subplots(1)
                ax.step(tsp1, np.hstack([us[0], us[0, -1]]), where='post')
                ax.set_ylim(-1.1, 1.1)
                fig.savefig(path + 'control_order_{}.png'.format(order), transparent=transparent)

                def plot_state_prediction(psi0, name):
                    psi0 = psi0 / psi0.norm()
                    rho0 = psi0.proj().full().reshape(-1, 1)
                    xs = [None] * ps.shape[1]
                    for i, p in enumerate(ps.T):
                        xs[i] = p.reshape((2 ** 2, 2 ** 2)) @ rho0
                    xs = np.hstack(xs)
                    fig, axes = plt.subplots(1, 4, figsize=[3 * 4 + 3, 3])
                    for i in range(4):
                        ax = axes[i]
                        ax.plot(tsp1, xs[i].real, ls='-', color=cmap(i))
                        ax.plot(tsp1, xs[i].imag, ls='--', color=cmap(i))
                        ax.set_ylim([-1.1, 1.1])
                    fig.tight_layout()
                    fig.savefig(path + '{}_order_{}.png'.format(name, order), transparent=transparent)

                # Example state preparations based on gate
                state = qt.basis(2, 0)
                plot_state_prediction(state, '1psi0')
                state = -3 * qt.basis(2, 1) + qt.basis(2, 0)
                plot_state_prediction(state, 'n3psi1_1psi0')

                fig, ax = plt.subplots(1)
                ax.plot(tsp1, [((p - pf).conj().T @ Q @ (p - pf)).real for p in ps.T])
                ax.set_ylim([1e-3, 1e1])
                ax.set_yscale('log')
                fig.savefig(path + 'cost_order_{}.png'.format(order), transparent=transparent)

# Run the test
TestGateSynth().test_NOT_gate()

Order: 1, Num Controls: 1
Expected polyu_dim: 1
Actual polyu_dim from N_op shape: 1


c:\ProgramData\anaconda3\envs\QuantumD\Lib\site-packages\qutip\solver\options.py:16: FutureWarning: Dedicated options class are no longer needed, options should be passed as dict to solvers.
  warnings.warn(


  0%|          | 0/50 [00:00<?, ?it/s]

c:\ProgramData\anaconda3\envs\QuantumD\Lib\site-packages\scipy\integrate\_ode.py:438: UserWarning: _zvode: Repeated convergence failures. (Perhaps bad Jacobian supplied or wrong choice of MF or tolerances.)
  self._y, self.t = mth(self.f, self.jac or (lambda: None),


IntegratorException: Repeated convergence failures. (Perhaps bad Jacobian supplied or wrong choice of MF or tolerances.)

In [ ]:
class TestStatePrep(TestCase):
    def test_crosstalk(self):
        # Simulation
        qubits = RWA_Crosstalk(0)

        # Basis (single qubit)
        measure_list = [qt.basis(2, i) * qt.basis(2, j).dag() for i in range(2) for j in range(2)]

        # Vectorize
        A_cts_list_1 = [m4q.vectorize_me(op, measure_list) for op in qubits.H_list_1]
        A_cts_list_2 = [m4q.vectorize_me(op, measure_list) for op in qubits.H_list_2]

        # Model
        clock = m4q.StepClock(dt=0.5, horizon=20, n_steps=50)
        sat = 2 * np.pi * 0.1
        du = 0.25
        clock.measure_freq = 2
        order = 1

        # Discretize
        A_cts_list = [block_diag(A_cts_list_1[0], A_cts_list_2[0])]
        n1, _ = A_cts_list_1[0].shape
        n2, _ = A_cts_list_2[0].shape
        for i in range(1, len(A_cts_list_2)):
            N2 = np.block([[np.zeros((n1, n1)), np.zeros((n1, n2))],
                           [np.zeros((n2, n1)), A_cts_list_2[i]]])
            A_cts_list.append(N2)
        for i in range(1, len(A_cts_list_1)):
            N1 = np.block([[A_cts_list_1[i], np.zeros((n1, n2))],
                           [np.zeros((n2, n1)), np.zeros((n2, n2))]])
            A_cts_list.append(N1)
        A_dst = m4q.discretize_homogeneous(A_cts_list, clock.dt, order)

        # DMD
        model_dim_x = 2 * 4
        dim_lift = len(m4q.create_library(order, qubits.dim_u)[1:])
        model1 = m4q.DMDc(model_dim_x, model_dim_x, dim_lift * model_dim_x, A_dst)

        # Objective
        Rx1 = qt.qip.operations.rx(-1e-3)
        Rx2 = qt.qip.operations.rx(1e-3)
        rho1_init = Rx1 * qt.basis(2, 0).proj() * Rx1.dag()
        rho2_init = Rx2 * qt.basis(2, 0).proj() * Rx2.dag()
        rho1_targ = qt.basis(2, 1).proj()
        rho2_targ = qt.basis(2, 0).proj()
        initial_state = qt.tensor(rho1_init, rho2_init).full().flatten()
        target_state = np.hstack([rho1_targ.full().flatten(), rho2_targ.full().flatten()])

        # Benchmarks
        X_bm = np.hstack([target_state[:, None]] * (clock.n_steps + clock.horizon + 1))
        U_bm = np.hstack([np.zeros((qubits.dim_u, 1))] * (clock.n_steps + clock.horizon))

        # Cost
        Q = block_diag(np.array([[1, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 1]]),
                       np.array([[1, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 1]]))
        qf_val = 1
        Qf = Q * qf_val
        r_val = 1e-3
        R = np.identity(qubits.dim_u) * r_val

        # MPC
        data, model2, exit_code = m4q.mpc(initial_state, qubits.dim_u, order, X_bm, U_bm, clock, qubits.QE, model1,
                                          Q, R, Qf, sat=sat, du=du, warm_start=False)

        xs, us = data
        xs_proj = np.vstack([qubits.QE.lift(xi) for xi in xs.T]).T
        x1 = xs_proj[:4, :]
        x2 = xs_proj[4:, :]
        ts = clock.ts_sim[::clock.measure_freq]
        tsplus1 = np.hstack([clock.ts_sim, clock.ts_sim[-1] + clock.dt])

        for xs, targ in zip([x1, x2], [rho1_targ, rho2_targ]):
            xs = np.atleast_2d(xs[:, :-1][:, ::clock.measure_freq])
            fig, axes = plt.subplots(2, 1, figsize=(6, 4))
            ax = axes[0]
            for row in xs:
                ax.plot(ts, row.real, marker='.', markerfacecolor='None')
            ax.set_ylim([-1.1, 1.1])

            ax = axes[1]
            infidelity = [1 - qt.fidelity(qt.Qobj(x.reshape(2, 2)), targ) for x in xs.T]
            ax.plot(ts, infidelity)
            ax.set_ylim([5e-4, 1.5])
            ax.set_yscale('log')
            
        for row in us:
            fig, ax = plt.subplots(1, figsize=(6, 2))
            ax.step(tsplus1, np.hstack([row, row[-1]]), where='post')
            max_u = np.max(np.abs(us))
            ax.set_ylim([-max_u * 1.1, max_u * 1.1])

        fig, axes = plot_operator(A_dst, 8)

# Run the tests
TestStatePrep().test_crosstalk()